Imports

In [7]:
import os
import re
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

Laddar in modell

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Bearbetar alla FAQ filer

In [9]:
faq_texts = []
questions = []

for filename in os.listdir('Data'):
    if filename.endswith(".txt"):
        with open(os.path.join('Data', filename), 'r', encoding='utf-8') as file:
            content = file.read().strip()

            match = re.search(r'FRÅGA:\s*(.*?)\n+ALTERNATIVA FRÅGOR:(.*?)\n+SVAR:\s*(.*)', content, re.DOTALL)
            if match:
                huvudfråga = match.group(1).strip()
                alternativa = match.group(2).strip().split('\n')
                svar = match.group(3).strip()
                alla_frågor = [huvudfråga] + [fråga.strip('- ').strip() for fråga in alternativa]

                for fråga in alla_frågor:
                    faq_texts.append(f"FRÅGA: {fråga}\nSVAR: {svar}")
                    questions.append(fråga)
            else:
                print(f"Misslyckades att parsa: {filename}")

Sparar text_entries.txt (som sedan används av chattboten)

In [10]:
with open("Data/text_entries.txt", "w", encoding="utf-8") as f:
    for entry in faq_texts:
        f.write(entry + "\n<END>\n")

Skapar embeddings och FAISS-index

In [11]:
embeddings = model.encode(questions, normalize_embeddings=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

Sparar FAISS-index

In [12]:
faiss.write_index(index, "Data/faiss_index.index")
print("Indexet har sparats som Data/faiss_index.index")

Indexet har sparats som Data/faiss_index.index
